In [ ]:
#LOAD DATA
%matplotlib inline
import matplotlib.pyplot as plt
from Quantapp.DataManager import DataManager
import plotly.express as px
import plotly.graph_objects as go

#import backtrader as bt
import numpy as np
import json
import os
plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams.update({'font.size': 12}) 
#https://www.backtrader.com/docu/quickstart/quickstart/

#from Quantapp.Portfolio  import Portfolio
#from Quantapp.Universe import  Universe
#from Quantapp.Algorithm import Algorithm
#from Quantapp.Computation import Computation


import pandas as pd
#algorithm = Algorithm()
dm        = DataManager()
#comp      = Computation()
period = 'max'
interval = '1d'

def plot_returns(series_returns):
    negative_returns = series_returns[series_returns<0]
    mean = negative_returns.mean()
    standard_deviation = negative_returns.std()
    standard_deviation_level_three_fourths = mean - .5 * standard_deviation
    standard_deviation_level_single        = mean - standard_deviation
    
    fig = px.line(series_returns)

    fig.add_hline(y=0, line_dash="dash", line_color="black", 
                annotation_text="Zero Line", annotation_position="bottom right")
    fig.add_hline(y=mean, line_dash="dot", line_color="blue", 
                annotation_text=f"Mean of negative returns: {mean:.2f}", annotation_position="top right")
    fig.add_hline(y=standard_deviation_level_three_fourths , line_dash="dashdot", line_color="red", 
                annotation_text=f".75 Std Dev: {standard_deviation_level_three_fourths :.2f}", annotation_position="top right")
    fig.add_hline(y=standard_deviation_level_single, line_dash="dashdot", line_color="red", 
                annotation_text=f"1 Std Dev: {standard_deviation_level_single:.2f}", annotation_position="top right")

    fig.add_shape(
        type="rect",
        x0=series_returns.index.min(),
        x1=series_returns.index.max(),
        y0=standard_deviation_level_three_fourths,
        y1=standard_deviation_level_single,
        fillcolor="green",
        opacity=0.2,
        line_width=0,
    )

    return fig

def create_spread_plot(asset_spreads):
    spread_threshold = 0
    spread           = asset_spreads
    mean             = spread[spread>=0].mean()
    std_dev = spread[spread >= 0].std()
    #spread = asset_spreads[:200]


    fig = px.line(spread)
    fig.update_layout(title=asset_spreads.name)
    fig.add_hline(y=spread_threshold, line_dash="dash", line_color="red", annotation_text=f"y={spread_threshold}")
    fig.add_hline(y=mean , line_color="red", annotation_text="mean")
    fig.add_hline(y=mean + std_dev, line_dash="dash", line_color="blue", 
                  annotation_text="mean + 1 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean - std_dev, line_dash="dash", line_color="blue", 
                  annotation_text="mean - 1 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean + 2*std_dev, line_dash="dot", line_color="green", 
                  annotation_text="mean + 2 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean - 2*std_dev, line_dash="dot", line_color="green", 
                  annotation_text="mean - 2 std dev", annotation_position="bottom right")
    fig.add_shape(type="rect",
                  xref="paper", yref="y",
                  x0=0, y0=mean+std_dev, x1=1, y1=spread.max(),
                  fillcolor="green", opacity=0.2, line_width=0)
    fig.update_layout(height=800)
    return fig



def add_labels(df):
    df['name of day'] = df.index.day_name()
    df['name of month'] = df.index.month_name()
    df['month/day'] = df.index.strftime('%m/%d') 
    return df

sp500 = dm.retrieve_ticker_data('SPY', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)
nasdaq = dm.retrieve_ticker_data('QQQ', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)
russell_2000 = dm.retrieve_ticker_data('IWM', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)
dija = dm.retrieve_ticker_data('DIA', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)

gold               = dm.retrieve_ticker_data('GLD', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)
treasury_bonds_20y = dm.retrieve_ticker_data('TLT', period=period,interval=interval).drop(['Dividends', 'Stock Splits','Capital Gains'],axis=1)

risk_free_rate   = dm.retrieve_ticker_data('^IRX',period=period,interval=interval).reindex_like(sp500)
vix   = dm.retrieve_ticker_data('^VIX',period=period).drop(['Dividends', 'Stock Splits'],axis=1)


In [ ]:
#Seasonality

sp500_monthly = sp500.resample('M').last()#.reset_index(inplace=True)
sp500_monthly_returns = sp500_monthly.pct_change() * 100

months = ['January', 'February','March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data = []
for i in range(12):
    data.append(sp500_monthly_returns[sp500_monthly_returns.index.month.isin([i+1])].mean()['Close'])

fig = px.bar(y=data, x =months, title='Seasonality: S&P 500')
fig.show()



nasdaq_monthly = nasdaq.resample('M').last()#.reset_index(inplace=True)
nasdaq_monthly_returns = nasdaq_monthly.pct_change() * 100

months = ['January', 'February','March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data = []
for i in range(12):
    data.append(nasdaq_monthly_returns[nasdaq_monthly_returns.index.month.isin([i+1])].mean()['Close'])

fig = px.bar(y=data, x =months, title='Seasonality: Nasdaq 100')
fig.show()


dija_monthly = dija.resample('M').last()#.reset_index(inplace=True)
dija_monthly_returns = dija_monthly.pct_change() * 100

months = ['January', 'February','March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data = []
for i in range(12):
    data.append(dija_monthly_returns[dija_monthly_returns.index.month.isin([i+1])].mean()['Close'])

fig = px.bar(y=data, x =months, title='Seasonality: DIJA')
fig.show()

vix_monthly = vix.resample('M').last()#.reset_index(inplace=True)
vix_monthly_returns = vix_monthly.pct_change() * 100

months = ['January', 'February','March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data = []
for i in range(12):
    data.append(vix_monthly_returns[vix_monthly_returns.index.month.isin([i+1])].mean()['Close'])

fig = px.bar(y=data, x =months, title='Seasonality: VIX')
fig.show()



In [ ]:
#Day by Data Statistics

sp500_daily= sp500#.resample('M').last()#.reset_index(inplace=True)
sp500_daily_returns = sp500_daily.pct_change() * 100
sp500_daily_returns['name of day'] = sp500_daily_returns.index.day_name()
sp500_daily_returns['name of month'] = sp500_daily_returns.index.month_name()
sp500_daily_returns['month/day'] = sp500_daily_returns.index.strftime('%m/%d') 
month_days = sp500_daily_returns['month/day'].unique()
name_of_days= sp500_daily_returns['name of day'].unique()
def convert_date(date):
    try:
        return pd.to_datetime(date, format='%m/%d')
    except ValueError:
        return None
    
converted_dates = [convert_date(x) for x in month_days]
converted_dates = [date for date in converted_dates if date is not None]
sorted_dates = sorted(converted_dates)
sorted_dates = [date.strftime('%m/%d') for date in sorted_dates]

data = []
for month_day in sorted_dates:
    data.append(sp500_daily_returns[sp500_daily_returns['month/day'] == month_day]['Close'].mean())


fig = px.bar(y=data, x=sorted_dates, title='Average Returns for Each Day of the Year')
fig.show()

data = []
for name_of_day in name_of_days:
    data.append(sp500_daily_returns[sp500_daily_returns['name of day'] == name_of_day]['Close'].mean())
fig = px.bar(y=data, x=name_of_days, title='Average Returns for Each Day')
fig.show()


monday = sp500_daily_returns[sp500_daily_returns['name of day'] == 'Monday']['Close']
fig = px.bar((monday>0).rolling(window=21).sum()/21, title='succesive monday returns')
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="green")
fig.add_hline(y=.6, line_width=1, line_dash="dash", line_color="red")
fig.show()

tuesday = sp500_daily_returns[sp500_daily_returns['name of day'] == 'Tuesday']['Close']
fig = px.bar((tuesday>0).rolling(window=21).sum()/21, title='succesive tuesday returns')
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="green")
fig.add_hline(y=.6, line_width=1, line_dash="dash", line_color="red")
fig.show()

wednesday = sp500_daily_returns[sp500_daily_returns['name of day'] == 'Wednesday']['Close']
fig = px.bar((wednesday>0).rolling(window=21).sum()/21, title='succesive wednesday returns')
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="green")
fig.add_hline(y=.6, line_width=1, line_dash="dash", line_color="red")
fig.show()

thursday = sp500_daily_returns[sp500_daily_returns['name of day'] == 'Thursday']['Close']
fig = px.bar((thursday>0).rolling(window=21).sum()/21, title='succesive thursday returns')
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="green")
fig.add_hline(y=.6, line_width=1, line_dash="dash", line_color="red")
fig.show()

friday = sp500_daily_returns[sp500_daily_returns['name of day'] == 'Friday']['Close']
fig = px.bar((friday>0).rolling(window=21).sum()/21, title='succesive friday returns')
fig.add_hline(y=.5, line_width=1, line_dash="dash", line_color="green")
fig.add_hline(y=.6, line_width=1, line_dash="dash", line_color="red")
fig.show()



In [ ]:
#Returns
time_frame = 200

sp500_returns_fig              = plot_returns(sp500['Close'].pct_change(time_frame))
sp500_returns_fig.update_layout(title='S&P 500 200 Day returns')\

russell_2000_returns_fig              = plot_returns(russell_2000['Close'].pct_change(time_frame))
russell_2000_returns_fig.update_layout(title='russell 2000 500 200 Day returns')

treasury_bonds_20y_returns_fig = plot_returns(treasury_bonds_20y['Close'].pct_change(time_frame))
treasury_bonds_20y_returns_fig.update_layout(title='20 year Treasury bonds 200D returns')

gold_returns_fig               = plot_returns(gold['Close'].pct_change(time_frame))
gold_returns_fig.update_layout(title='Gold 200 day returns')

sp500_returns_fig.show()
russell_2000_returns_fig.show()
treasury_bonds_20y_returns_fig.show()
gold_returns_fig.show()


In [ ]:
#Relative Returns 
time_frame = 200

russell_2000_returns_fig              = create_spread_plot(sp500['Close'].pct_change(time_frame) - russell_2000['Close'].pct_change(time_frame))
russell_2000_returns_fig.update_layout(title='russell 2000 500 200 Day returns')

treasury_bonds_20y_returns_fig = create_spread_plot(sp500['Close'].pct_change(time_frame) - treasury_bonds_20y['Close'].pct_change(time_frame))
treasury_bonds_20y_returns_fig.update_layout(title='20 year Treasury bonds 200D returns')

gold_returns_fig               = create_spread_plot(sp500['Close'].pct_change(time_frame) - gold['Close'].pct_change(time_frame))
gold_returns_fig.update_layout(title='Gold 200 day returns')

russell_2000_returns_fig.show()
treasury_bonds_20y_returns_fig.show()
gold_returns_fig.show()